In [1]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
import astroalign as aa

import subprocess
import os
import sys
sys.path.append("../../Resolved_Polarization_v3")
sys.path.append("../../analysis_v3")
from resolved_pol import ResolvedPol

In [2]:
#Load the image with the worse seeing from W0204, as everything is being matched to it. 
fors2_im = fits.open("../../analysis_v3/crz/science_reduced_img.W0204-0506.60209.R_SPECIAL.chip1.3.crz.fits")
emask = fits.getdata("../../analysis_v3/masks/science_reduced_img.W0204-0506.60209.R_SPECIAL.chip1.3.emask.fits",dtype=bool)

In [3]:
fits.writeto("W0204-0506.FORS2.ebeam.fits",np.where(emask, np.nan, fors2_im[0].data), overwrite=True)

In [4]:
#Load the F160W and F555W images which are already aligned to each other.
f555w = fits.open("id1j03010_drc_sci.ali.smoothed.fits")
f160w = fits.open("id1j03020_drz.fits")

In [5]:
#It is too difficult to do this automatically, so we'll just use a few stars we have selected by hand. The centroids are eye-balled, so this can certainly be improved.
data = np.loadtxt("stars.txt")
src_f160w = list()
dst = list()
for i in range(len(data)):
    src_f160w.append((data[i,0]-1, data[i,1]-1))
    dst.append((data[i,2]-1, data[i,3]-1))
src_f160w = np.array(src_f160w)
dst = np.array(dst)

In [6]:
#Now, we need to get the pixel positions in the F555W image, which has a different number of pixels than the F160W image. 
WCS_F160W = WCS(f160w['SCI'].header)
WCS_F555W = WCS(f555w[0].header)
src_f555w = list()
for i in range(len(src_f160w)):
    sky = WCS_F160W.pixel_to_world(src_f160w[i][0], src_f160w[i][1])
    x, y = WCS_F555W.world_to_pixel(sky)
    src_f555w.append((x,y))
src_f555w = np.array(src_f555w)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [7]:
#Run astroalign to match the hst image to the fors2 image using the calculated transformation.
target_masked = np.ma.array(fors2_im[0].data, mask=emask)
target_masked = target_masked.byteswap().newbyteorder()

#F160W
source = f160w['SCI'].data
source = source.byteswap().newbyteorder()
tform = aa.estimate_transform('affine',src_f160w,dst)
registered_f160w, footprint = aa.apply_transform(tform, source, target_masked)

#F555W
source = f555w[0].data
source = source.byteswap().newbyteorder()
tform = aa.estimate_transform('affine',src_f555w,dst)
registered_f555w, footprint = aa.apply_transform(tform, source, target_masked)

In [8]:
#So now we need to call the ResolvedPol object just to match exactly the default stamp size used. 
bhd = ResolvedPol("W0204-0506", "R_SPECIAL", star_coords=(1086.5, 222.5), root_folder=os.getcwd()+"/../../analysis_v3")
_ = subprocess.call("rm -r work stamps", shell=True)

mkdir: work: File exists


In [9]:
#Make the stamps. 
f555w_stamp = registered_f555w[bhd.iy1_z:bhd.iy2_z, bhd.ix1_z:bhd.ix2_z]
f160w_stamp = registered_f160w[bhd.iy1_z:bhd.iy2_z, bhd.ix1_z:bhd.ix2_z]

In [10]:
fits.writeto("f555w.stamp.fits", f555w_stamp, overwrite=True)
fits.writeto("f160w.stamp.fits", f160w_stamp, overwrite=True)